<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/_gp_bart_model_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [3]:
import torch
import numpy as np

## CPU
device = torch.device("cpu")

## GPU
# device = torch.device("cuda")

In [4]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=ccd1fc3cc2abe7839da04a66d74730241be999f5491f6b59392e9554e981ed68
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [5]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 145 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ou0gursf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ou0gursf
     |████████████████████████████████| 129 kB 5.1 MB/s 
     |████████████████████████████████| 54.7 MB 7.9 MB/s 
     |████████████████████████████████| 4.5 MB 30.1 MB/s 
     |████████████████████████████████| 1.2 MB 50.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3a920000 @  0x7fa9f40e7615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 

In [7]:
# 사용자 input

# 서버에서 본문 받아오기
# 서버에서 아마 csv나 json으로 받아올텐데, 받아올 때 특수문자 처리 필수 (특히 따옴표 '', "")
# '[제목] 본문' 형태로 전처리
# //////////////////////서버로부터 본문 받아와서 str로 저장하는 line////////////////////////
input_text= "[아르바이트]한 달에 오 만원 정도의 용돈을 받아서 절반은 교통카드 충전을, \
또 절반은 친구들과 노는 곳에 주로 썼었다. 이 패턴은 고등학교를 다닐 때에도 마찬가지였다. \
한 달에 십 만원 정도의 용돈을 받아서 절반은 교통카드 충전 또 절반은 친구들과 노는 곳에 썼었지만 \
확실히 고등학교 때에는 돈이 모자르게만 느껴졌었고 아르바이트를 통해 부족한 돈을 충당하기보다는 \
버스를 타는 것 대신 삼십여분거리를 걸어서 등하교를 하고 그 교통비를 용돈에 보태서 쓰는 방법을 택했었다. \
어릴 때부터 용돈 받는선 일에 겁도 많이 났었지만 일단 한 번 해보자. 해봐야 뒷일도 아는 거니까라는 마음으로 \
첫 출근을 해서 일 년간 꾸준히 빵집 아르바이트를 통해 처음 저축도 하고 부모님께 선물도 사 드리고 \
내가 입고싶었던 옷, 핸드폰도 구입하고 보람찬 일들을 많이 했었기 때문에 \
아직도 그 빵집 앞을 지나갈 때나 혹은 익숙한 빵들을 보고 있을 때면 괜히 그때의 일들이 생각나 마음이 붕뜨고 설렌 적도 많았던 것 같다. \
부모님께 의존하고 부모님이 해주시는 모든 것들을 당연하게만 느꼈었던 과거의 나에게 지금의 내가 한마디 해줄 수 있다면 \
네가 받고 네가 누리고 있는 모든 것들에 항상 감사한 마음을 가지라고 해주고 싶다. \
별거 아니네라고 치부하며 낭비하지 말고 작은 것도 소중히 여겨 아낄 줄 아는 마음을 가진 내가 되었으면 좋겠다."


In [8]:
if len(input_text) + len(" TL;DR ") > 800:
  input_text = input_text[0:400] + input_text[len(input_text)-400:len(input_text)]

* BERT

In [9]:
from kobert import get_pytorch_kobert_model

bertmodel, vocab  = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
import gluonnlp as nlp
from kobert import get_tokenizer

bert_tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(bert_tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
from torch import nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [12]:
bert_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [13]:
# model load
bert_model.load_state_dict(torch.load('./checkpoint/bert_model.pt', map_location=device))

<All keys matched successfully>

In [14]:
max_len = 512

In [15]:
# 기쁨:0 분노:1 혐오:2 두려움:3 슬픔:4 unknown:5

def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["기쁨", "분노", "혐오", "두려움", "슬픔", "unknown"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("일기의 감정:", cate[idx])
    print("신뢰도:", "{:.2f}%".format(softmax(result,idx)))
    return cate[idx]

In [20]:
# torch.__version__
# !pip show transformers

Name: transformers
Version: 4.8.1
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: huggingface-hub, pyyaml, packaging, tqdm, filelock, requests, numpy, sacremoses, importlib-metadata, tokenizers, regex
Required-by: kobert


In [21]:
emotion= testModel(bert_model, input_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


일기의 감정: 기쁨
신뢰도: 90.09%


* KoBART

In [22]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

In [23]:
model = BartForConditionalGeneration.from_pretrained('./KoBART-summarization/logs/model_chp')

In [24]:
if len(input_text) > 800:
  input_text = input_text[0:400] + input_text[len(input_text)-400:len(input_text)]

In [25]:
text= emotion + '| ' + input_text

text = text.replace('\n', ' ')
# print(text)

In [26]:
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
output= tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [27]:
print(output)

아르바이트를 통해 부족한 돈을 충당하기보다는 버스를 타는 것 대신 삼십여분거리를 걸어서 등하교를 하고 그 교통비를 용돈에 보태서 쓰는 방법을 택했군요. 분명 좋은 결과가 있을 거에요!
